<a href="https://colab.research.google.com/github/leohafsa/5thB_Assignmnt/blob/master/ML_unets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install scipy==1.1.0
! pip install pillow

     |████████████████████████████████| 31.2MB 149kB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [ ]:
!pip install SimpleITK

     |████████████████████████████████| 44.9MB 104kB/s 


In [ ]:
cd /content/drive/My Drive/unidata/StrokeSegmentation-master

/content/drive/My Drive/unidata/StrokeSegmentation-master


In [ ]:
! python Pre_Processing.py

/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site8//031948//t02//
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site8//031950//t02//
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site8//031957//t02//
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site8//031959//t02//
/content/drive/My Drive/unidata/StrokeSegmentation-master/ATLAS_R1.1/Site8//031972//t02//


In [ ]:
from __future__ import division, print_function

from keras import backend as K
K.set_image_data_format('channels_last')
import numpy as np
from scipy.ndimage import morphology

import SimpleITK as sitk
from skimage.exposure import equalize_adapthist, equalize_hist
#import cv2
import math
import random
from skimage import measure
from scipy import ndimage

smooth=1.0

def Avoid_out_of_range(Center_0,size_0,leng_0):
    if Center_0-size_0/2<0:
        Center_0=size_0/2+1

    if Center_0+size_0/2>leng_0:
        Center_0=leng_0-size_0/2-1
    new_center=int(Center_0)
    return new_center

def smooth_images(imgs, t_step=0.125, n_iter=5):
    """
    Curvature driven image denoising.
    In my experience helps significantly with segmentation.
    """
    for mm in range(len(imgs)):
        img = sitk.GetImageFromArray(imgs[mm])
        img = sitk.CurvatureFlow(image1=img,
                                        timeStep=t_step,
                                        numberOfIterations=n_iter)

        imgs[mm] = sitk.GetArrayFromImage(img)
    return imgs

def preprocess(imgs):
    """add one more axis as tf require"""
    imgs = imgs[..., np.newaxis]
    return imgs

def preprocess_front(imgs):
    imgs = imgs[np.newaxis, ...]
    return imgs

def img_resize(imgs, img_rows, img_cols, equalize=True):

    new_imgs = np.zeros([len(imgs), img_rows, img_cols])
    for mm, img in enumerate(imgs):
        if equalize:
            img = equalize_adapthist( img, clip_limit=0.05 )

        new_imgs[mm] = cv2.resize( img, (img_rows, img_cols), interpolation=cv2.INTER_NEAREST )

    return new_imgs

def imgs_adapthist(imgs):
    new_imgs = np.zeros([len(imgs),imgs.shape[1],imgs.shape[2]])
    for mm, img in enumerate(imgs):
        img = equalize_adapthist( img, clip_limit=0.05 )
        new_imgs[mm] = img

    return new_imgs

def Find_rangeZ(GS_3D):
    max_Z = np.zeros([GS_3D.shape[0]])
    for i in range(GS_3D.shape[0]):
        max_Z[i]=np.max(GS_3D[i,:,:])
        #print(max_Z[i])
    arr = np.nonzero(max_Z)
    start_idx = min(arr[0])
    end_idx = max(arr[0])+1
    return start_idx, end_idx

def itk_to_array(image_filename):
    image_itk = sitk.ReadImage(image_filename)
    image_array = sitk.GetArrayFromImage(image_itk)
    return image_array

def Label_connection(prob_):
    area_list = []
    binary = np.zeros(prob_.shape,dtype=np.uint8)
    binary[prob_>0.5]=1
    #binary = prob_ > 0.5
    label_prob_ = measure.label(binary)
    region_label_prob_ = measure.regionprops(label_prob_)
    for region in region_label_prob_:
        area_list.append(region.area)
    idx_max = np.argmax(area_list)
    binary[label_prob_ != idx_max + 1] = 0
    temp_prob_ = binary
    #temp_prob_.astype(np.uint8)
    #plt.figure('Orginal Prob'), plt.imshow(temp_prob_, cmap='gray')
    #plt.show()
    return temp_prob_
'''
def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (
        K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
'''

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1.-dice_coef(y_true, y_pred)

#focal loss
def focal_loss(y_true, y_pred, gamma=2.0):
    y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
    eps = K.epsilon()
    y_pred = K.clip(y_pred, eps, 1. - eps)
    loss = -K.sum(K.pow(1. - y_pred, gamma) * y_true * K.log(y_pred), axis=-1)
    return loss

def numpy_dice(y_true, y_pred, axis=None, smooth=1.0):

    intersection = y_true*y_pred

    return ( 2. * intersection.sum(axis=axis) +smooth)/ (np.sum(y_true, axis=axis) + np.sum(y_pred, axis=axis) +smooth )


def rel_abs_vol_diff(y_true, y_pred):

    return np.abs( (y_pred.sum()/y_true.sum() - 1)*100)


def get_boundary(data, img_dim=2, shift = -1):
    data  = data>0
    edge = np.zeros_like(data)
    for nn in range(img_dim):
        edge += ~(data ^ np.roll(~data,shift=shift,axis=nn))
    return edge.astype(int)



def surface_dist(input1, input2, sampling=1, connectivity=1):
    input1 = np.squeeze(input1)
    input2 = np.squeeze(input2)

    input_1 = np.atleast_1d(input1.astype(np.bool))
    input_2 = np.atleast_1d(input2.astype(np.bool))


    conn = morphology.generate_binary_structure(input_1.ndim, connectivity)

    S = input_1 - morphology.binary_erosion(input_1, conn)
    Sprime = input_2 - morphology.binary_erosion(input_2, conn)


    dta = morphology.distance_transform_edt(~S,sampling)
    dtb = morphology.distance_transform_edt(~Sprime,sampling)

    sds = np.concatenate([np.ravel(dta[Sprime!=0]), np.ravel(dtb[S!=0])])

    return sds

def Find_ranges(mask,margin_xy,margin_z):
    #print(np.shape(mask))
    arr=np.nonzero(mask)
    Min_z=int(min(arr[0])-margin_z)
    Max_z=int(max(arr[0])+margin_z)

    Min_x=int(min(arr[1])-margin_xy)
    Max_x=int(max(arr[1])+margin_xy)
    Min_y=int(min(arr[2])-margin_xy)
    Max_y=int(max(arr[2])+margin_xy)
    
    Min_z=max(0,Min_z)
    Max_z=min(np.shape(mask)[0],Max_z)

    return Min_z,Max_z,Min_x,Max_x,Min_y,Max_y

In [ ]:
! pip install metrics

     |████████████████████████████████| 849kB 4.3MB/s 
  Created wheel for metrics: filename=metrics-0.3.3-py2.py3-none-any.whl size=17815 sha256=8976e9e7c905d3ba3a112c5fc1ddd068ca6106e7883f5352fb754229a86a0ada
  Stored in directory: /root/.cache/pip/wheels/38/e8/55/c4c648451a33a60920318eba4c5735088dbf78dc20d4277db0
  Created wheel for pathspec: filename=pathspec-0.5.5-cp36-none-any.whl size=24229 sha256=aeef6c1f2d2ea4b8bbef273a480c9856984f3f125d44f323c3d563408dffc26e
  Stored in directory: /root/.cache/pip/wheels/d1/2e/6b/c67842adf3b13ea33ca3fe6a2a963622821ad647db724688e2
Successfully built metrics pathspec
ERROR: jupyterlab-pygments 0.1.2 has requirement pygments<3,>=2.4.1, but you'll have pygments 2.2.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1


In [ ]:
! python data.py

5
031950t02_IMGAG.nii
031950t02
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031950t02_IMGAG.nii
031957t02_IMGAG.nii
031957t02
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031957t02_IMGAG.nii
031959t02_IMGAG.nii
031959t02
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031959t02_IMGAG.nii
031972t02_IMGAG.nii
031972t02
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031972t02_IMGAG.nii
(199, 233, 197, 1)


In [ ]:
! python loss.py

2020-11-03 09:58:18.667214: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
! python model_Unet_pading.py

2020-11-03 09:56:52.290830: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-11-03 09:56:54.070873: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-03 09:56:54.135038: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-11-03 09:56:54.135093: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (6dcc76380c74): /proc/driver/nvidia/version does not exist
2020-11-03 09:56:54.166100: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2250000000 Hz
2020-11-03 09:56:54.166432: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x17e2a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-11-03 09:56:54.166470: I tensorflow/com

In [ ]:
! python train_Unet_main.py

2020-11-03 09:58:41.307354: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
5
031950t02_IMGAG.nii
031950t02
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031950t02_IMGAG.nii
031957t02_IMGAG.nii
031957t02
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031957t02_IMGAG.nii
031959t02_IMGAG.nii
031959t02
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031959t02_IMGAG.nii
031972t02_IMGAG.nii
031972t02
/content/drive/My Drive/unidata/StrokeSegmentation-master/PP_data/Image//031972t02_IMGAG.nii
(199, 233, 197, 1)
(199, 233, 197, 1)
Data augmentation was NOT used
Validation was used
(199, 233, 197, 2)
2020-11-03 09:58:47.084052: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-11-03 09:58:47.097196: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_E

In [ ]:
! python test_Unet_main.py